<a href="https://colab.research.google.com/github/Shahzad-kml/ML1/blob/main/Anomalies_Detection_(Muhammad_Shahzad).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Importing the required libraries**

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

**Loading the data of Inverter**

In [ ]:
data= pd.read_csv('/content/Inverter Data Set.csv')
print(data.isnull().sum())
print(data.dtypes)
print(data.corr())

n_k         0
u_dc_k      0
u_dc_k-1    0
u_dc_k-2    0
u_dc_k-3    0
i_a_k       0
i_b_k       0
i_c_k       0
i_a_k-1     0
i_b_k-1     0
i_c_k-1     0
i_a_k-2     0
i_b_k-2     0
i_c_k-2     0
i_a_k-3     0
i_b_k-3     0
i_c_k-3     0
d_a_k-2     0
d_b_k-2     0
d_c_k-2     0
d_a_k-3     0
d_b_k-3     0
d_c_k-3     0
u_a_k-1     0
u_b_k-1     0
u_c_k-1     0
dtype: int64
n_k         float64
u_dc_k      float64
u_dc_k-1    float64
u_dc_k-2    float64
u_dc_k-3    float64
i_a_k       float64
i_b_k       float64
i_c_k       float64
i_a_k-1     float64
i_b_k-1     float64
i_c_k-1     float64
i_a_k-2     float64
i_b_k-2     float64
i_c_k-2     float64
i_a_k-3     float64
i_b_k-3     float64
i_c_k-3     float64
d_a_k-2     float64
d_b_k-2     float64
d_c_k-2     float64
d_a_k-3     float64
d_b_k-3     float64
d_c_k-3     float64
u_a_k-1     float64
u_b_k-1     float64
u_c_k-1     float64
dtype: object
               n_k    u_dc_k  u_dc_k-1  u_dc_k-2  u_dc_k-3     i_a_k  \
n_k       1.00000

**Filling the missing values using mean**

In [ ]:
data['d_c_k-2'].fillna(data['d_c_k-2'].mean(), inplace= True)
data['d_a_k-3'].fillna(data['d_a_k-3'].mean(), inplace= True)
data['d_b_k-3'].fillna(data['d_b_k-3'].mean(), inplace= True)
data['d_c_k-3'].fillna(data['d_c_k-3'].mean(), inplace= True)
data['u_a_k-1'].fillna(data['u_a_k-1'].mean(), inplace= True)
data['u_b_k-1'].fillna(data['u_b_k-1'].mean(), inplace= True)
data['u_c_k-1'].fillna(data['u_c_k-1'].mean(), inplace= True)

**Spliting the dataset into training testing and Training a model using Ensemble learning package (Iforest) Tree based model**

In [ ]:
data_for_training=data.iloc[10000:,:]
data_for_testing=data.iloc[:10000,:]
data_for_training.shape, data_for_testing.shape
n_features=len(data.columns)
from sklearn.ensemble import IsolationForest
iforest=IsolationForest(n_estimators=100, max_samples='auto', contamination=float(0.2),max_features=n_features)
iforest.fit(data_for_training)
scores=iforest.decision_function(data_for_training)
predicted=iforest.predict(data_for_training)
score=iforest.decision_function(data_for_testing)

**Checking the prediction**

In [ ]:
data_for_testing['prediction']=iforest.predict(data_for_testing)
testingData=data_for_testing.copy()
testingData['Score']=pd.DataFrame(score)


**Checking the total rows which are marked as anomaly**

In [ ]:
anomaly=testingData.loc[testingData.prediction==-1]
anomaly_index=list(anomaly.index)
print(anomaly)
print(len(anomaly_index))

              n_k      u_dc_k    u_dc_k-1    u_dc_k-2    u_dc_k-3     i_a_k  \
15    3002.750389  568.908235  569.018988  568.871318  568.465225  1.112132   
16    3002.832698  568.760565  568.908235  569.018988  568.871318  0.996429   
17    3002.919047  568.649812  568.760565  568.908235  569.018988  0.870626   
18    3003.000067  568.723648  568.649812  568.760565  568.908235  0.733804   
41    3003.111363  567.394616  567.320781  567.394616  567.837627 -3.123854   
...           ...         ...         ...         ...         ...       ...   
9995  2999.744473  561.155552  560.638706  560.601789  560.454119  3.247113   
9996  2999.567226  561.672398  561.155552  560.638706  560.601789  3.178242   
9997  2999.386188  561.893903  561.672398  561.155552  560.638706  3.057031   
9998  2999.210807  562.004655  561.893903  561.672398  561.155552  2.871540   
9999  2999.031871  561.967738  562.004655  561.893903  561.672398  2.632789   

         i_b_k     i_c_k   i_a_k-1   i_b_k-1  ...  

**Making a cluster using K-means clustering**

In [ ]:
from pycaret.clustering import *
training_data=data.iloc[10000:,:]
testing_data=data.iloc[:10000,:]
model2=setup(training_data, normalize=True)
kmeans=create_model('kmeans') # (This line taking a time so I interrupted this cell of code)
plot_model(kmeans,plot='cluster') # (This line of code us a graph with 4 cluster where we can easily distinguish between anomalies and other datapoints)

,Description,Value
0,Session id,8412
1,Original data shape,"(224527, 26)"
2,Transformed data shape,"(224527, 26)"
3,Numeric features,26
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,Normalize,True
9,Normalize method,zscore


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:15:36
Status,. . . . . . . . . . . . . . . . . .,Fitting 4 Clusters
Estimator,. . . . . . . . . . . . . . . . . .,K-Means Clustering


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

**Using Pycaret library and its anomaly detection package and setting up environment**

In [ ]:
from sklearn import model_selection
from pycaret.anomaly import *
model4=setup(training_data)

,Description,Value
0,Session id,3656
1,Original data shape,"(224527, 26)"
2,Transformed data shape,"(224527, 26)"
3,Numeric features,26
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


In [ ]:
# create Model
iforest = create_model('iforest')
print(iforest)

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

IForest(behaviour='new', bootstrap=False, contamination=0.05,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
    random_state=3656, verbose=0)


In [ ]:
# Plotting a graph 
plot_model(iforest, plot='tsne') # This of line of code give us a 3D graph to analyse the datapoints with differnt color schemes.

KeyboardInterrupt: ignored

In [ ]:
# assigning a model
result=assign_model(iforest)
result.head()

,n_k,u_dc_k,u_dc_k-1,u_dc_k-2,u_dc_k-3,i_a_k,i_b_k,i_c_k,i_a_k-1,i_b_k-1,...,d_b_k-2,d_c_k-2,d_a_k-3,d_b_k-3,d_c_k-3,u_a_k-1,u_b_k-1,u_c_k-1,Anomaly,Anomaly_Score
0,2998.858887,561.893921,561.967712,562.004639,561.893921,2.362817,2.158516,-4.461905,2.632789,1.936366,...,0.955271,0.479381,0.137688,0.952312,0.430591,58.537758,520.008484,285.079346,1,0.009541
1,2998.682373,561.746216,561.893921,561.967712,562.004639,2.102947,2.339014,-4.394381,2.362817,2.158516,...,0.955726,0.527883,0.134729,0.955271,0.479381,58.800236,519.835449,312.385834,1,0.008508
2,2998.511963,561.783142,561.746216,561.893921,561.967712,1.821037,2.496370,-4.268583,2.102947,2.339014,...,0.953817,0.576572,0.134274,0.955726,0.527883,59.767677,518.237549,339.701111,1,0.007947
3,2998.338623,561.930847,561.783142,561.746216,561.893921,1.528109,2.620404,-4.102086,1.821037,2.496370,...,0.950659,0.624467,0.136183,0.953817,0.576572,62.114876,516.729614,366.446716,0,-0.001232
4,2998.162109,562.078491,561.930847,561.783142,561.746216,1.207632,2.684272,-3.901364,1.528109,2.620404,...,0.945138,0.670571,0.139342,0.950659,0.624467,66.071754,513.822144,392.483856,0,-0.012502


In [ ]:
# Making prediction
prediction=predict_model(iforest, data=testing_data)
prediction.head()

,n_k,u_dc_k,u_dc_k-1,u_dc_k-2,u_dc_k-3,i_a_k,i_b_k,i_c_k,i_a_k-1,i_b_k-1,...,d_b_k-2,d_c_k-2,d_a_k-3,d_b_k-3,d_c_k-3,u_a_k-1,u_b_k-1,u_c_k-1,Anomaly,Anomaly_Score
0,3001.406296,567.985297,567.689956,567.431534,567.948379,2.461991,-1.792057,-0.716639,2.729208,-2.098439,...,0.874633,0.125367,0.706206,0.862754,0.137246,360.541201,510.640963,82.471117,0,-0.065788
1,3001.468250,567.911462,567.985297,567.689956,567.431534,2.292110,-1.556948,-0.757338,2.461991,-1.792057,...,0.880046,0.119954,0.667181,0.874633,0.125367,346.410081,513.190083,79.729357,0,-0.061216
2,3001.527815,567.911462,567.911462,567.985297,567.689956,2.155288,-1.332946,-0.840587,2.292110,-1.556948,...,0.884307,0.115693,0.642184,0.880046,0.119954,329.440240,515.804643,77.924637,0,-0.067000
3,3001.585080,567.653039,567.911462,567.911462,567.985297,2.048768,-1.135788,-0.925686,2.155288,-1.332946,...,0.888915,0.111085,0.611911,0.884307,0.115693,311.058880,517.537603,75.478717,0,-0.065363
4,3001.640131,567.579204,567.653039,567.911462,567.911462,1.952350,-0.918266,-1.027434,2.048768,-1.135788,...,0.892123,0.107877,0.578149,0.888915,0.111085,290.481760,519.138403,73.580077,0,-0.068689


In [ ]:
#Total anomalies are detected
anomaly2=prediction.loc[prediction.Anomaly==1]
anomaly_index2=list(anomaly2.index)
print(anomaly2)
print(len(anomaly_index2))

              n_k      u_dc_k    u_dc_k-1    u_dc_k-2    u_dc_k-3     i_a_k  \
91    2999.989144  567.542286  567.689956  567.726874  567.800709  4.164466   
92    2999.947596  568.096049  567.542286  567.689956  567.726874  4.948670   
93    2999.911560  567.616121  568.096049  567.542286  567.689956  5.549219   
94    2999.880732  567.025441  567.616121  568.096049  567.542286  5.920201   
95    2999.854820  567.542286  567.025441  567.616121  568.096049  6.180072   
...           ...         ...         ...         ...         ...       ...   
9987  3004.417335  573.855185  573.855185  574.039773  573.965938  4.853169   
9988  3004.186461  573.596763  573.855185  573.855185  574.039773  4.895410   
9989  3003.959588  573.301422  573.596763  573.855185  573.855185  4.896328   
9991  3003.500249  573.301422  573.116835  573.301422  573.596763  4.784299   
9999  2999.031871  561.967738  562.004655  561.893903  561.672398  2.632789   

         i_b_k     i_c_k   i_a_k-1   i_b_k-1  ...  